In [ ]:
# 웹툰 각 img url 지정 다운로드

import requests
import os

req_header = {
  # Image를 가져오기 위한 Header (어디서 가져오는지)
  'referer': 'https://comic.naver.com/webtoon/list?titleId=833255&tab=tue',
}

img_urls = [
  'https://image-comic.pstatic.net/webtoon/833255/50/20250311161718_49d9b68af514fc9e2ee85feb5edccb12_IMAG01_1.jpg',
  'https://image-comic.pstatic.net/webtoon/833255/50/20250311161718_49d9b68af514fc9e2ee85feb5edccb12_IMAG01_2.jpg',
  'https://image-comic.pstatic.net/webtoon/833255/50/20250311161718_49d9b68af514fc9e2ee85feb5edccb12_IMAG01_3.jpg'
]

for img_url in img_urls:
    # requets.get(url, headers) : url에 요청을 보내고 응답을 받는 함수
    res = requests.get(img_url, headers=req_header)
    # 응답 상태 코드 출력 (200이면 성공)
    print(res.status_code)

    if res.ok:
        # 응답 데이타를 바이트(Binary)로 가져옴
        img_data = res.content
        # 파일 이름을 URL에서 추출
        file_name = os.path.basename(img_url)
        # Binary 모드로 파일을 열고, 응답 데이터를 저장
        with open(file_name, '+wb') as file:
            file.write(img_data)
            print(f"Image saved as {file_name}({len(img_data):,} bytes)")

In [ ]:
# 웹툰 img 전체 다운로드 (BeautifulSoup 사용)

import requests
from bs4 import BeautifulSoup
import os

url = 'https://comic.naver.com/webtoon/detail?titleId=183559&no=653&week=finish'
req_header = {
    'referer': url
}
res = requests.get(url)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')  # jpg 파일의 absolute path url 찾기
    print(len(soup.select("img[src*='IMAG01']")))  # IMAG01가 포함된 img src 찾기
    print(len(soup.select("img[src$='.jpg']")))  # jpg로 끝나는 img src 찾기
    print(len(soup.select("img[src^='https://image-comic']")))  # https://image-comic로 시작하는 img src 찾기

    img_tags = soup.select("img[src*='IMAG01']")
    # img src를 저장할 list 선언
    img_url_list = []

    for img_tag in img_tags:
        # print(type(img_tag), img_tag.name, img_tag)
        img_src = img_tag['src']
        img_url_list.append(img_src)  # img src를 list에 추가
    print(len(img_url_list), img_url_list[:2])  # img src list의 len 출력

    # img src를 저장할 디렉토리 생성 (없으면)
    imgdir_name = 'img'
    if not os.path.isdir(imgdir_name):
        os.mkdir(imgdir_name)

    for img_url in img_url_list:
        # requets.get(url, headers) : url에 요청을 보내고 응답을 받는 함수
        res_img = requests.get(img_url, headers=req_header)
        # 응답 상태 코드 출력 (200이면 성공)
        print(res_img.status_code)

        if res_img.ok:
            img_data = res_img.content
            file_name = os.path.basename(img_url)
            save_path = os.path.join(imgdir_name, file_name)
            with open(save_path, '+wb') as file:
                file.write(img_data)
                print(f"Image saved as {save_path} ({len(img_data):,} bytes)")
        else:
            print(f"Error downloading image from {img_url}: {res_img.status_code}")
else:
    print('Error: ', res.status_code)
    exit(1)

In [ ]:
# 2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기 (필수)

import requests
from bs4 import BeautifulSoup
import os

# 웹툰 제목과 회차 번호, url 지정
title = '별이삼샵'
no = '200'
url = 'https://comic.naver.com/webtoon/detail?titleId=737628&no=201&week=sun'
req_header = {
    'referer': url
}

def download_one_episode(title,no,url):
    res = requests.get(url)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')  # jpg 파일의 absolute path url 찾기
        print(len(soup.select("img[src*='IMAG01']")))  # IMAG01가 포함된 img src 찾기
        print(len(soup.select("img[src$='.jpg']")))  # jpg로 끝나는 img src 찾기
        print(len(soup.select("img[src^='https://image-comic']")))  # https://image-comic로 시작하는 img src 찾기
    
        img_tags = soup.select("img[src*='IMAG01']")
        # img src를 저장할 list 선언
        img_url_list = []
    
        for img_tag in img_tags:
            # print(type(img_tag), img_tag.name, img_tag)
            img_src = img_tag['src']
            img_url_list.append(img_src)  # img src를 list에 추가
        print(len(img_url_list), img_url_list[:2])  # img src list의 len 출력
    
        # img src를 저장할 디렉토리 생성 (없으면)
        imgdir_img = 'img'
        imgdir_title_name = f"{title}"
        imgdir_no_name = f"{no}"
        imgdir_name = os.path.join(imgdir_img, imgdir_title_name, imgdir_no_name)

        # 상위 디렉토리가 없는 경우 먼저 생성
        os.makedirs(imgdir_name, exist_ok=True)
    
        for img_url in img_url_list:
            # requets.get(url, headers) : url에 요청을 보내고 응답을 받는 함수
            res_img = requests.get(img_url, headers=req_header)
            # 응답 상태 코드 출력 (200이면 성공)
            print(res_img.status_code)
    
            if res_img.ok:
                img_data = res_img.content
                file_name = os.path.basename(img_url)
                save_path = os.path.join(imgdir_name, file_name)
                with open(save_path, 'wb') as file:
                    file.write(img_data)
                    print(f"Image saved as {save_path} ({len(img_data):,} bytes)")
            else:
                print(f"Error downloading image from {img_url}: {res_img.status_code}")
    else:
        print('Error: ', res.status_code)
        exit(1)

download_one_episode(title,no,url)

97
307
308
97 ['https://image-comic.pstatic.net/webtoon/737628/201/20250111221606_1978638f433e51f8708f3b457df5bd35_IMAG01_1.jpg', 'https://image-comic.pstatic.net/webtoon/737628/201/20250111221606_1978638f433e51f8708f3b457df5bd35_IMAG01_2.jpg']
200
Image saved as img\별이삼샵\200\20250111221606_1978638f433e51f8708f3b457df5bd35_IMAG01_1.jpg (116,228 bytes)
200
Image saved as img\별이삼샵\200\20250111221606_1978638f433e51f8708f3b457df5bd35_IMAG01_2.jpg (211,777 bytes)
200
Image saved as img\별이삼샵\200\20250111221606_1978638f433e51f8708f3b457df5bd35_IMAG01_3.jpg (223,716 bytes)
200
Image saved as img\별이삼샵\200\20250111221606_1978638f433e51f8708f3b457df5bd35_IMAG01_4.jpg (164,259 bytes)
200
Image saved as img\별이삼샵\200\20250111221606_1978638f433e51f8708f3b457df5bd35_IMAG01_5.jpg (47,916 bytes)
200
Image saved as img\별이삼샵\200\20250111221606_1978638f433e51f8708f3b457df5bd35_IMAG01_6.jpg (57,480 bytes)
200
Image saved as img\별이삼샵\200\20250111221606_1978638f433e51f8708f3b457df5bd35_IMAG01_7.jpg (202,589 b

In [ ]:
# 2-3. 여러 개의 네이버 웹툰과 회차에 대한 Image 다운로드 하기 (선택)

import requests
from bs4 import BeautifulSoup
import os

# 웹툰 제목과 url 지정
title = '별이삼샵'
titleId = '737628'
episode_url = f'https://comic.naver.com/webtoon/list?titleId={titleId}'
req_header = {
    'referer': episode_url
}

def download_all_episode(title, episode_url):
    res = requests.get(episode_url, headers=req_header)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        # detail 페이지의 url 패턴을 정확하게 매칭하도록 수정
        href_list = soup.select("a[href*='/webtoon/detail?titleId=737628&no=']")
        print("Extracted href attributes:", href_list)

        episode_number_set = set()
        for href in href_list:
            link = href.get('href')
            if link:
                parts = link.split('&')
                for part in parts:
                    if part.startswith('no='):
                        episode_number = part.split('=')[1]
                        episode_number_set.add(episode_number)
                        break

        print("Extracted episode numbers:", sorted(list(episode_number_set), reverse=True))

    else:
        print("Error fetching episode_url:", res.status_code)
    # download_one_episode(title,no,url)

download_all_episode(title, episode_url)

def download_one_episode(title,no,url):
    res = requests.get(url)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')  # jpg 파일의 absolute path url 찾기    
        img_tags = soup.select("img[src*='IMAG01']")  # IMAG01가 포함된 img src 찾기
        # img src를 저장할 list 선언
        img_url_list = []
    
        for img_tag in img_tags:
            # print(type(img_tag), img_tag.name, img_tag)
            img_src = img_tag['src']
            img_url_list.append(img_src)  # img src를 list에 추가
        print(len(img_url_list), img_url_list[:2])  # img src list의 len 출력
    
        # img src를 저장할 디렉토리 생성 (없으면)
        imgdir_img = 'img'
        imgdir_title_name = f"{title}"
        imgdir_no_name = f"{no}"
        imgdir_name = os.path.join(imgdir_img, imgdir_title_name, imgdir_no_name)

        # 상위 디렉토리가 없는 경우 먼저 생성
        os.makedirs(imgdir_name, exist_ok=True)
    
        for img_url in img_url_list:
            # requets.get(url, headers) : url에 요청을 보내고 응답을 받는 함수
            res_img = requests.get(img_url, headers=req_header)
            # 응답 상태 코드 출력 (200이면 성공)
            print(res_img.status_code)
    
            if res_img.ok:
                img_data = res_img.content
                file_name = os.path.basename(img_url)
                save_path = os.path.join(imgdir_name, file_name)
                with open(save_path, 'wb') as file:
                    file.write(img_data)
                    print(f"Image saved as {save_path} ({len(img_data):,} bytes)")
            else:
                print(f"Error downloading image from {img_url}: {res_img.status_code}")
    else:
        print('Error: ', res.status_code)
        exit(1)

Extracted href attributes: []
Extracted episode numbers: []
